# Assignment description

Build synthetic scale-free networks according to the BA model. Consider $m$ new connections for each node inserted. Calculate empiracally properties like CC and APL. Compare the obtained results for networks of same size generated by Poisson model and WS model. What do you observe?

In [30]:
%config Completer.use_jedi = False

In [31]:
import igraph as ig
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Graph parameters

We're going to build some Erdos-Renyi (ER) models varying its parameters. When we fix a connection probability we're also fixing its degree. So we'll create some realizations for each combination of n and z.

In [32]:
nodes_array = np.arange(1000, 12000, 2000)#[1000, 2000, 4000, 6000, 8000, 10000]
avg_degree_array = [0, 0.5, 0.9, 1, 2, 3, 4, "2log"] #np.arange(0, 3.2, 0.2)
ensemble_size = 2

In [33]:
len(nodes_array) * len(avg_degree_array ) * ensemble_size

96

In [34]:
def find_percolation_threshold(n = 1000, threshold = 0.5, step = 0.2):
    graphs_dict = dict()
    avg_degree = 0
    while True:
        nodes = n
        graphs_dict[avg_degree] = dict()

        probability = avg_degree/nodes        
        graph = ig.Graph.Erdos_Renyi(n = nodes, p = probability)
        graphs_dict[avg_degree][probability] = graph
        giant = graph.clusters().giant()
        graphs_dict[avg_degree][probability]["giant"] = giant
        giant_fraction = giant.vcount()/n
        avg_degree += step
        if giant_fraction > threshold:
            return graphs_dict

percolations = find_percolation_threshold()

In [35]:
percolations

{0: {0.0: <igraph.Graph at 0x7fe072e1ed50>},
 0.2: {0.0002: <igraph.Graph at 0x7fe076ef1150>},
 0.4: {0.0004: <igraph.Graph at 0x7fe076ef1350>},
 0.6000000000000001: {0.0006000000000000001: <igraph.Graph at 0x7fe076ef1550>},
 0.8: {0.0008: <igraph.Graph at 0x7fe076ef1750>},
 1.0: {0.001: <igraph.Graph at 0x7fe076ef1950>},
 1.2: {0.0012: <igraph.Graph at 0x7fe076ef1b50>},
 1.4: {0.0014: <igraph.Graph at 0x7fe076ef1d50>},
 1.5999999999999999: {0.0015999999999999999: <igraph.Graph at 0x7fe079615050>}}

## Graph building

### Poisson networks

In [36]:
# %%time
# A Python program to print all
# permutations using library function
from itertools import product

# Get all permutations of nodes and probabilities
perm = product(*[nodes_array, avg_degree_array])

poisson_graphs = dict()
# Print the obtained permutations
for network_property in list(perm):
    
    graphs_list = list()
    nodes = int(network_property[0])
    
    if network_property[1] in ["log", "2log"]:
        degree = 2*np.round(np.log(nodes), 2)
        key = str(nodes) + '_nodes__' + network_property[1] + "_degree"
    else:
        degree = network_property[1]
        key = str(nodes) + '_nodes__' + str(degree) + "_degree"
    probability = degree/nodes
    
    # Create a few realizations
    for i in range(ensemble_size):
        graph = ig.Graph.Erdos_Renyi(n = nodes, p = probability)
        graphs_list.append(graph)
    
    poisson_graphs[key] = dict()
    poisson_graphs[key]['graphs'] = graphs_list
    poisson_graphs[key]["nodes"] = nodes
    poisson_graphs[key]["degree"] = network_property[1]
    poisson_graphs[key]["probability"] = probability    

In [37]:
%%time
# Calculate some metrics for each graph realization

nodes_number = list()
edges_number = list()
clustering_coefficient = list()
avg_path_length = list()

for params in poisson_graphs.keys():
    for graph in poisson_graphs[params]["graphs"]:
        clustering_coefficient.append(graph.transitivity_undirected())
        avg_path_length.append(graph.average_path_length())
        edges_number.append(len(graph.get_edgelist()))
        nodes_number.append(poisson_graphs[params]["nodes"])        

CPU times: user 1min 27s, sys: 260 ms, total: 1min 27s
Wall time: 1min 28s


In [38]:
poisson_analysis = pd.DataFrame()

poisson_analysis["nodes_number"] = nodes_number
poisson_analysis["edges_number"] = edges_number
poisson_analysis["clustering_coefficient"] = clustering_coefficient
poisson_analysis["avg_path_length"] = avg_path_length

poisson_analysis.head()

,nodes_number,edges_number,clustering_coefficient,avg_path_length
0,1000,0,NaN,NaN
1,1000,0,NaN,NaN
2,1000,260,0.0,1.776876
3,1000,262,0.0,1.831068
4,1000,427,0.0,5.075444


In [39]:
df_plot = poisson_analysis.pivot_table(index = ["nodes_number", "edges_number"],
                                         values = ['avg_path_length', "clustering_coefficient"],
                                         aggfunc = ["mean"])
df_plot

mean                       
                          avg_path_length clustering_coefficient
nodes_number edges_number                                       
1000         260                 1.776876               0.000000
             262                 1.831068               0.000000
             427                 5.075444               0.000000
             461                 4.088530               0.000000
             504                10.158257               0.000000
...                                   ...                    ...
11000        16741               8.301953               0.000351
             21778               6.880669               0.000348
             22058               6.820812               0.000238
             102055              3.518225               0.001636
             102061              3.518907               0.001828

[84 rows x 2 columns]

### Barabasi-Albert networks

In [40]:
nodes_array = np.arange(1000, 12000, 2000)#[1000, 2000, 4000, 6000, 8000, 10000]
ensemble_size = 2
m_number = np.arange(1, 6, 1)

In [41]:
%%time
# A Python program to print all
# permutations using library function
from itertools import product

# Get all permutations of nodes and probabilities
perm = product(*[nodes_array, m_number])

barabasi_graphs = dict()
# Print the obtained permutations
for network_property in list(perm):
    
    graphs_list = list()
    nodes = int(network_property[0])
    m_value = int(network_property[1])
    
    # Create a few realizations
    for i in range(ensemble_size):
        graph = ig.Graph.Barabasi(n = nodes, m = m_value)

        graphs_list.append(graph)

    key = str(nodes) + "_nodes__" + str(m_value) + "_m"
    barabasi_graphs[key] = dict()
    barabasi_graphs[key]['graphs'] = graphs_list
    barabasi_graphs[key]["nodes"] = nodes
    barabasi_graphs[key]["m"] = m_value

CPU times: user 248 ms, sys: 13.5 ms, total: 261 ms
Wall time: 266 ms


In [42]:
%%time
clustering_coefficient = list()
avg_path_length = list()
nodes_number = list()
edges_number = list()
m_values = list()

for params in barabasi_graphs.keys():
    for graph in barabasi_graphs[params]["graphs"]:
        clustering_coefficient.append(graph.transitivity_undirected())
        avg_path_length.append(graph.average_path_length())
        nodes_number.append(barabasi_graphs[params]["nodes"])
        m_values.append(barabasi_graphs[params]["m"]) 
        edges_number.append(len(graph.get_edgelist()))
        

CPU times: user 1min 42s, sys: 123 ms, total: 1min 43s
Wall time: 1min 43s


In [43]:
barabasi_analysis = pd.DataFrame()
barabasi_analysis["nodes_number"] = nodes_number
barabasi_analysis["m_values"] = m_values
barabasi_analysis["avg_path_length"] = avg_path_length
barabasi_analysis["clustering_coefficient"] = clustering_coefficient

barabasi_analysis.fillna(0, inplace = True)

In [44]:
df_plot = barabasi_analysis.pivot_table(index = ["nodes_number"],
                                         values = ['avg_path_length', "clustering_coefficient"],
                                         aggfunc = ["mean"])
df_plot

mean                       
             avg_path_length clustering_coefficient
nodes_number                                       
1000                4.432790               0.016987
3000                5.072445               0.007347
5000                5.315933               0.004777
7000                5.572558               0.003484
9000                5.856569               0.002871
11000               5.831589               0.002498

### Watts-Strogatz networks

In [45]:
nodes_array = np.arange(1000, 12000, 2000)#[1000, 2000, 4000, 6000, 8000, 10000]

ensemble_size = 2
steps_number = np.arange(1, 6, 1)
rewiring_probabilities = np.arange(0, 0.3, 0.05)

In [50]:
%%time
# A Python program to print all
# permutations using library function
from itertools import product

# Get all permutations of nodes and probabilities
perm = product(*[nodes_array, steps_number, rewiring_probabilities])

watts_strogatz_graphs = dict()
# Print the obtained permutations
for network_property in list(perm):
    
    graphs_list = list()
    nodes = int(network_property[0])
    steps = int(network_property[1])
    rewiring = int(network_property[2])
    
    
    # Create a few realizations
    for i in range(ensemble_size):
        graph = ig.Graph.Watts_Strogatz(dim = 1, size = nodes, nei = steps, p = rewiring)

        graphs_list.append(graph)

    key = str(nodes) + "_nodes__" + str(steps) + "_nei__" + str(p) + "_rewiring_prob"
    watts_strogatz_graphs[key] = dict()
    watts_strogatz_graphs[key]['graphs'] = graphs_list
    watts_strogatz_graphs[key]["nodes"] = nodes
    watts_strogatz_graphs[key]["rewiring_prob"] = rewiring

CPU times: user 699 ms, sys: 51.2 ms, total: 751 ms
Wall time: 758 ms


In [55]:
%%time
clustering_coefficient = list()
avg_path_length = list()
nodes_number = list()
edges_number = list()
rewiring_prob_values = list()

for params in watts_strogatz_graphs.keys():
    for graph in watts_strogatz_graphs[params]["graphs"]:
        clustering_coefficient.append(graph.transitivity_undirected())
        avg_path_length.append(graph.average_path_length())
        nodes_number.append(watts_strogatz_graphs[params]["nodes"])
        rewiring_prob_values.append(watts_strogatz_graphs[params]["rewiring_prob"]) 
        edges_number.append(len(graph.get_edgelist()))
        
watts_strogatz_analysis = pd.DataFrame()
watts_strogatz_analysis["nodes_number"] = nodes_number
watts_strogatz_analysis["rewiring_prob"] = rewiring_prob_values
watts_strogatz_analysis["avg_path_length"] = avg_path_length
watts_strogatz_analysis["clustering_coefficient"] = clustering_coefficient

watts_strogatz_analysis.fillna(0, inplace = True)

CPU times: user 46.9 s, sys: 44 ms, total: 47 s
Wall time: 47 s


In [57]:
watts_strogatz_analysis.shape

(60, 4)

In [58]:
poisson_analysis.shape

(96, 4)

In [59]:
barabasi_analysis.shape

(60, 4)